In [1]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#トークナイザとBERTモデルをロード、モデルをGPUに載せる
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
#BERTを文章穴埋めに応用したネットワーク
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 

In [3]:
#文章の一部を特殊トークン[MASK]に置き換える
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [4]:
#文章を符号化し、GPUに配置する
input_ids = tokenizer.encode(text,return_tensors='pt')
input_ids = input_ids.cuda()

#BERTに入力し、分類スコアを得る
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [5]:
#5-6
#ID列で'[MASK]'の位置を調べる
mask_position = input_ids[0].tolist().index(4)

#スコアが最も良いトークンのIDを取り出し、トークンに変換する
id_best = scores[0,mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##','')

#[MASK]を求めたトークンで置き換える
text = text.replace('[MASK]',token_best)

print(text)

今日は東京へ行く。


In [6]:
#上位10位のトークンに置き換える
def predict_mask_topk(text,tokenizer,bert_mlm,num_topk):
    """
    文章中の[MASK]をスコアの上位のトークンに置き換える
    上位何位まで使うかは、num_topkで指定
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト
    """
    #文章を符号化し、BERTで分類スコアを得る
    input_ids = tokenizer.encode(text,return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits
    
    #スコアが上位のトークンとスコアを求める
    mask_position = input_ids[0].tolist().index(4)
    topk = scores[0,mask_position].topk(num_topk)
    ids_topk = topk.indices#トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
    scores_topk = topk.values.cpu().numpy() #スコア
    
    #文章中の[MASK]を上で求めたトークンで置き換える
    text_topk = []#穴埋めされたテキストを追加
    for token in tokens_topk:
        token = token.replace('##','')
        text_topk.append(text.replace('[MASK]',token,1))
    
    return text_topk,scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text,tokenizer,bert_mlm,10)
print(*text_topk,sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [7]:
def greedy_prediction(text,tokenizer,bert_mlm):
    """
    [MASK]を含む文章を入力して、貧欲法で穴埋めを行った文章を出力する
    """
    #前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える
    for _ in range(text.count('[MASK]')):
        text  = predict_mask_topk(text,tokenizer,bert_mlm,1)[0][0]
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text,tokenizer,bert_mlm)

'今日は、東京へ行く。'

In [8]:
text = '今日は[MASK][MASK][MASK][MASK]'
greedy_prediction(text,tokenizer,bert_mlm)

'今日は社会的にも'

In [10]:
def beam_search(text,tokenizer,bert_mlm,num_topk):
    """
    ビームサーチで文章の穴埋めを行う
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        #現在得られている、それぞれの文章に対して、最初の[MASK]をスコアが上位のトークンで穴埋めする
        text_candidates = [] #それぞれの文章を穴埋めした結果を追加する
        score_candidates = []#穴埋めに使ったトークンのスコアを追加する
        for text_mask, score in zip(text_topk,scores_topk):
            text_topk_inner,scores_topk_inner = predict_mask_topk(
                text_mask,tokenizer,bert_mlm,num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append(score + scores_topk_inner)
            
        #穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort() [::-1] [:num_topk]
        text_topk = [text_candidates[idx] for idx in idx_list]
        scores_topk = score_candidates[idx_list]
    
    return text_topk

text = '今日は[MASK][MASK]へ行く。'
text_topk = beam_search(text,tokenizer,bert_mlm,10)
print(*text_topk,sep='\n')

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。
